In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
import scipy.signal as signalpy
import wave
import sys
import math
import contextlib
import IPython
import os
import librosa
import librosa.display
import pandas as pd
import sklearn.datasets as datasets
import shutil
import scipy
from scipy.interpolate import interp1d
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from keras.utils import to_categorical


### Signals as different columns

In [37]:
# Loading dataframe with signals saved as separate columns

pcg_dataframe_separated = pd.DataFrame(data = np.load('arrays/pch_dataframe_separated.npy' , allow_pickle=True))



In [3]:
y = pcg_dataframe_separated[2] # Murmur presence/absence only = labels
display(y)
y.value_counts()


0       0
1       0
2       0
3       0
4       1
       ..
3002    0
3003    0
3004    0
3005    0
3006    0
Name: 2, Length: 3007, dtype: object

2
0    2391
1     616
Name: count, dtype: int64

In [16]:
# Checking for unknown values

# y = y.astype(str)
# unknown_values = y[y.str.contains("unknown")]

# print(unknown_values)


[0 1]


In [4]:
x = pcg_dataframe_separated
#IT HAS TO BE RUN ONLY ONCE
x.drop(columns = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16] , inplace=True) # Dropping all columns except signal
display(x)

,17,18,19,20,21,22,23,24,25,26,...,27480,27481,27482,27483,27484,27485,27486,27487,27488,27489
0,0.044284,0.083573,0.084831,0.079909,0.076818,0.079682,0.063184,0.074145,0.061855,0.056708,...,0.027397,0.024591,0.013935,-0.045194,-0.067835,-0.034756,-0.009343,-0.022132,0.036019,0.0
1,0.037838,0.060296,0.055494,0.062987,0.06844,0.080645,0.070724,0.075249,0.049619,0.032656,...,-0.022082,-0.023812,-0.155682,0.097941,0.136763,-0.002966,-0.074278,-0.061395,-0.032471,0.0
2,0.188557,0.042485,-0.017912,0.01128,-0.023341,0.007332,-0.015273,0.005513,-0.01603,0.008373,...,-0.025098,0.011518,0.262761,-0.178561,-0.470868,-0.382222,-0.083927,0.282585,0.230621,0.144693
3,0.006233,-0.005921,-0.016664,0.006667,-0.006036,-0.002884,-0.005222,0.003192,-0.004282,0.001617,...,-0.013384,-0.015804,-0.021826,-0.027921,-0.022429,-0.013681,0.006725,0.020161,0.036498,0.045997
4,-0.100144,-0.823798,-0.67462,-0.455151,-0.366007,-0.242825,-0.21219,-0.128745,-0.132289,-0.060894,...,-0.041863,-0.03375,-0.024635,-0.02028,-0.014789,-0.013059,-0.010073,-0.009028,-0.00695,-0.006936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3002,0.006546,0.017596,0.003999,0.022795,0.020831,0.070317,0.02719,-0.008061,-0.013101,-0.040103,...,0.020862,0.038278,0.003789,-0.029651,-0.00628,-0.018723,-0.011551,-0.033721,0.023264,0.0
3003,0.03042,0.008324,-0.021918,-0.022625,0.008196,0.012661,-0.013639,-0.02529,-0.014853,-0.009143,...,-0.011914,-0.013201,-0.003142,-0.001335,-0.000031,-0.003905,-0.003665,-0.00648,-0.003663,0.0
3004,0.002058,-0.00899,0.002498,0.007698,0.011565,0.018659,0.02825,0.021532,0.026256,0.01901,...,0.04229,-0.034819,0.045834,-0.041132,0.023717,-0.013726,0.021793,0.005888,0.197115,-0.056327
3005,0.074213,0.059252,0.005898,-0.053307,-0.127206,-0.100906,-0.051948,0.018652,0.040873,-0.007306,...,-0.011283,0.049323,-0.125811,-0.026233,0.040654,0.31941,-0.082078,-0.02337,0.008176,0.0


In [43]:
# display(x)

# I want to parse the x dataframe and check if there is any "unknown" value in it


# for index, row in x.iterrows():
#     for col in row.index:
#         if row[col] == "unknown":
#             print(f"unknown value found at index: {index}, column: {col}")


,17,18,19,20,21,22,23,24,25,26,...,27480,27481,27482,27483,27484,27485,27486,27487,27488,27489
0,0.044284,0.083573,0.084831,0.079909,0.076818,0.079682,0.063184,0.074145,0.061855,0.056708,...,0.027397,0.024591,0.013935,-0.045194,-0.067835,-0.034756,-0.009343,-0.022132,0.036019,0.0
1,0.037838,0.060296,0.055494,0.062987,0.06844,0.080645,0.070724,0.075249,0.049619,0.032656,...,-0.022082,-0.023812,-0.155682,0.097941,0.136763,-0.002966,-0.074278,-0.061395,-0.032471,0.0
2,0.188557,0.042485,-0.017912,0.01128,-0.023341,0.007332,-0.015273,0.005513,-0.01603,0.008373,...,-0.025098,0.011518,0.262761,-0.178561,-0.470868,-0.382222,-0.083927,0.282585,0.230621,0.144693
3,0.006233,-0.005921,-0.016664,0.006667,-0.006036,-0.002884,-0.005222,0.003192,-0.004282,0.001617,...,-0.013384,-0.015804,-0.021826,-0.027921,-0.022429,-0.013681,0.006725,0.020161,0.036498,0.045997
4,-0.100144,-0.823798,-0.67462,-0.455151,-0.366007,-0.242825,-0.21219,-0.128745,-0.132289,-0.060894,...,-0.041863,-0.03375,-0.024635,-0.02028,-0.014789,-0.013059,-0.010073,-0.009028,-0.00695,-0.006936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3002,0.006546,0.017596,0.003999,0.022795,0.020831,0.070317,0.02719,-0.008061,-0.013101,-0.040103,...,0.020862,0.038278,0.003789,-0.029651,-0.00628,-0.018723,-0.011551,-0.033721,0.023264,0.0
3003,0.03042,0.008324,-0.021918,-0.022625,0.008196,0.012661,-0.013639,-0.02529,-0.014853,-0.009143,...,-0.011914,-0.013201,-0.003142,-0.001335,-0.000031,-0.003905,-0.003665,-0.00648,-0.003663,0.0
3004,0.002058,-0.00899,0.002498,0.007698,0.011565,0.018659,0.02825,0.021532,0.026256,0.01901,...,0.04229,-0.034819,0.045834,-0.041132,0.023717,-0.013726,0.021793,0.005888,0.197115,-0.056327
3005,0.074213,0.059252,0.005898,-0.053307,-0.127206,-0.100906,-0.051948,0.018652,0.040873,-0.007306,...,-0.011283,0.049323,-0.125811,-0.026233,0.040654,0.31941,-0.082078,-0.02337,0.008176,0.0


In [5]:
x_test, x_train, y_test, y_train = train_test_split(x, y, test_size=0.25, stratify=y)

In [7]:
# print(np.unique(y))
# print(np.unique(y_train))
# print(np.unique(y_test))
# print(type(y[0]))
# print(type(y_train[0]))
# print(type(y_test[1]))

# print(y_test.shape)
# # print(y_test[0].dtype)
# print(type(y_test[6]))

# print(type(y_test.iloc[3]))

print("Indexes of y_test:", y_test.index.tolist())
print("Indexes of y_train:", y_train.index.tolist())

train_indexes = set(y_train.index)
test_indexes = set(y_test.index)

print("Train indexes:", len(train_indexes))
print("Test indexes:", len(test_indexes))

intersection = train_indexes.intersection(test_indexes)
print("Intersection:", len(intersection))

if len(intersection) > 0:
    print("Error: Some indexes appear in both train and test sets")
    
if len(train_indexes) + len(test_indexes) != len(y):
    print("Error: Some indexes are missing")

Indexes of y_test: [2910, 1558, 2979, 466, 2103, 1872, 1542, 995, 1960, 1021, 2361, 1220, 1500, 2803, 750, 215, 205, 1120, 1133, 171, 24, 1966, 1623, 1916, 385, 2629, 1314, 2092, 1179, 2978, 2911, 2038, 604, 1518, 1594, 2770, 229, 2371, 75, 2997, 1168, 543, 2285, 717, 2224, 2769, 2782, 1611, 2692, 1993, 2908, 2369, 1416, 100, 2496, 1658, 2032, 441, 1640, 1975, 1731, 2074, 2156, 2685, 1901, 602, 988, 1339, 1337, 1028, 1250, 1489, 2675, 1832, 2819, 2696, 2621, 546, 894, 421, 129, 1570, 1231, 1584, 870, 618, 571, 2996, 1331, 2742, 1806, 2737, 429, 2866, 93, 2517, 1831, 562, 2832, 1748, 424, 468, 787, 1644, 730, 2125, 2164, 59, 1453, 739, 1984, 1842, 2090, 1110, 2115, 1793, 386, 513, 2645, 2584, 2356, 291, 831, 1156, 2213, 1836, 1654, 483, 2215, 192, 2666, 745, 1126, 343, 2370, 2308, 2615, 1843, 2040, 1686, 2192, 2898, 1789, 1323, 2276, 580, 2467, 2783, 1562, 864, 2316, 2303, 2801, 695, 1209, 1366, 1934, 1121, 352, 1833, 41, 2935, 1672, 945, 887, 819, 319, 892, 691, 1980, 353, 1606, 2529, 

In [6]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

classifier = KNeighborsClassifier(n_neighbors=1)

classifier.fit(x_train, y_train)

train_accuracy = classifier.score(x_train, y_train)
test_accuracy = classifier.score(x_test, y_test)

In [7]:
#report
print(f"Train Accuracy: {(train_accuracy*100).round(2)}%")
print(f"Test Accuracy: {(test_accuracy*100).round(2)}%")

classifier.get_params()

Train Accuracy: 100.0%
Test Accuracy: 21.95%


{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 1,
 'p': 2,
 'weights': 'uniform'}

In [8]:
y_pred = classifier.predict(x_test)

In [9]:
# I want to send the side by side comparison to a csv file

input_dir = 'prediction_comparison/'
if not os.path.exists(input_dir):
    os.makedirs(input_dir)


with open(input_dir + 'KNN_predictions.csv', 'w') as f:
    for i in range(len(y_test)):
        f.write("Correct murmur attributes: " + str(y_test[i]) + ", ")
        f.write("Predicted murmur attributes: " + str(y_pred[i]) + "\n")


### Conclusions : all the predicted values are 1.0 . I am not sure why.